# LSTM con Keras

Le reti **Long short-term memory (LSTM)** sono un'architettura di reti neurali ricorrenti che risolvono il problema della scomparsa del gradiente tra le diverse esecuzioni della rete.
<br>
Le LSTM  sono state da Sepp Hochreiter e Jurger Schmidhuber nel 1997 in [questa ricerca](https://www.bioinf.jku.at/publications/older/2604.pdf) ed in sostanza aggiungono uno stato alla rete (cell state) che viaggia in parallelo al segnale della rete e immagazzina le informazioni sequenziali.
<br><br>
In questo notebook utilizzeremo le LSTM per migliorare la rete neurale per la sentiment analysis addestrata sull'IMDB Movie Reviews dataset.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

## Scarichiamo il dataset
Utilizziamo Keras per caricare l'imdb dataset, limitandolo alle 10000 parole più comuni.

In [4]:
from keras.datasets import imdb 

num_words = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


## Preprocessiamo i dati
Le recensioni all'interno del corpus di testo hanno ovviamente lunghezza differente, utilizziamo la funzione pad_sequences di keras per limitare le sequenze a 500 elementi (nel nostro caso limitare le frasi a 500 parole).
Se una sequenza ha meno di 500 esempi verranno aggiunti degli zeri alla fine.


In [5]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 500

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

X_train.shape

(25000, 500)

## Creiamo il modello
Costruiamo il nostro modello strutturandolo così:
1. Il primo strato eseguirà l'embedding creando 50 embedding vectors per ognuna delle 10.000 parole nel nostro dizionario.
2. Il secondo strato è lo strato ricorrente di tipo Long-short term memory.
3. Il terzo strato calcolerà l'ouput della rete, trattandosi di un problema di classifcazione binaria (recensione positiva/negativa) la funzione di attivazione sarà la sigmoide.

In [0]:
from keras.layers import Embedding, LSTM

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Compiliamo il modello ed eseguiamo l'addestramento per 5 epoche.

In [0]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=5)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 38s 2ms/step - loss: 0.6105 - acc: 0.6824 - val_loss: 0.4930 - val_acc: 0.8076
Epoch 2/10
20000/20000 [==============================] - 37s 2ms/step - loss: 0.4157 - acc: 0.8399 - val_loss: 0.7061 - val_acc: 0.6286
Epoch 3/10
14336/20000 [====================>.........] - ETA: 9s - loss: 0.3363 - acc: 0.8712 

Il risultato è nettamente migliore rispetto ad una semplice RNN, c'è un po' di overfitting, proviamo a correggerlo utilizzando il dropout.

## Dropout in una RNN
In una RNN possiamo utilizzare due differenti tipologie di dropout.
1. Dropout sull'input/output dello strato, esattamente come abbiamo già fatto con le altre architetture di reti neurali.
2. Dropout tra le esecuzioni ricorrenti della rete, questo permette di ridurre l'overfitting nelle features che contengono le informazioni sulla sequenza.

Per utilizzare il dropout sull'input di uno strato ricorrente, piuttosto che usare la classe Dropout, è consigliato sfruttare il parametro dropout delle classi SimpleRNN e LSTM.
Per utilizzare il dropout ricorrente possiamo invece utilizzare il parametro recurrent_dropout delle classi SimpleRNN e LSTM.

In [13]:
from keras.layers import Embedding, LSTM, Dropout

model = Sequential()

model.add(Embedding(num_words, 50))
model.add(LSTM(32, dropout=0.4))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 50)          500000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per 10 epoche.

In [14]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=20)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 40s 2ms/step - loss: 0.6331 - acc: 0.6621 - val_loss: 0.5072 - val_acc: 0.8122
Epoch 2/20
20000/20000 [==============================] - 38s 2ms/step - loss: 0.4384 - acc: 0.8278 - val_loss: 0.4537 - val_acc: 0.8064
Epoch 3/20
20000/20000 [==============================] - 39s 2ms/step - loss: 0.3428 - acc: 0.8691 - val_loss: 0.3549 - val_acc: 0.8458
Epoch 4/20
20000/20000 [==============================] - 39s 2ms/step - loss: 0.2886 - acc: 0.8903 - val_loss: 0.3398 - val_acc: 0.8578
Epoch 5/20
20000/20000 [==============================] - 39s 2ms/step - loss: 0.2543 - acc: 0.9035 - val_loss: 1.6610 - val_acc: 0.5480
Epoch 6/20
20000/20000 [==============================] - 38s 2ms/step - loss: 0.2558 - acc: 0.9109 - val_loss: 0.3010 - val_acc: 0.8802
Epoch 7/20
20000/20000 [==============================] - 38s 2ms/step - loss: 0.2162 - acc: 0.9211 - val_loss: 0.2813 - val_acc:

KeyboardInterrupt: ignored

Abbiamo ridotto l'overfitting nella nostra rete, adesso proviamo a migliorare il modello aggiungendo degli altri strati ricorrenti alla rete.

## Aggiungiamo altri strati LSTM
Possiamo aggiungere altri strati ricorrenti ad una rete nella solita maniera, utilizzando il metodo add della classe Sequential.
Tieni conto solo di una cosa, di default la classe LSTM esegue il flattening della sequenza per poterla dare come input ad uno strato denso, possiamo modificare tale comportamento impostando il parametro return_sequences a True.

In [0]:
from keras.layers import Embedding, LSTM

model = Sequential()

model.add(Embedding(num_words, 100))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(64, dropout=0.4, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 100)         1000000   
_________________________________________________________________
lstm_13 (LSTM)               (None, None, 32)          17024     
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 1,041,921
Trainable params: 1,041,921
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per 10 epoche.

In [0]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10)
model.evaluate(X_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 85s 4ms/step - loss: 0.5919 - acc: 0.6885 - val_loss: 0.4701 - val_acc: 0.7732
Epoch 2/10
 1536/20000 [=>............................] - ETA: 1:13 - loss: 0.4208 - acc: 0.8190

KeyboardInterrupt: ignored